In [64]:
from importlib import reload
# import VAE as VAE
import VAE.DataLoader as DataLoader
import VAE.Encoder as Encoder
import VAE.Decoder as Decoder
import VAE.ConvSeq2Seq as ConvSeq2Seq
import VAE.Discriminator as Discriminator
import torch
from torch import nn
import math

In [2]:
dloader = DataLoader.DataLoader(50, 25, './data/h3.6m/dataset')

Reading subject 1, action walking, subaction 1
Reading subject 1, action walking, subaction 2
Reading subject 1, action eating, subaction 1
Reading subject 1, action eating, subaction 2
Reading subject 1, action smoking, subaction 1
Reading subject 1, action smoking, subaction 2
Reading subject 1, action discussion, subaction 1
Reading subject 1, action discussion, subaction 2
Reading subject 1, action directions, subaction 1
Reading subject 1, action directions, subaction 2
Reading subject 1, action greeting, subaction 1
Reading subject 1, action greeting, subaction 2
Reading subject 1, action phoning, subaction 1
Reading subject 1, action phoning, subaction 2
Reading subject 1, action posing, subaction 1
Reading subject 1, action posing, subaction 2
Reading subject 1, action purchases, subaction 1
Reading subject 1, action purchases, subaction 2
Reading subject 1, action sitting, subaction 1
Reading subject 1, action sitting, subaction 2
Reading subject 1, action sittingdown, subacti

In [3]:
encoder_data, discriminator_data, yhat = dloader.get_train_batch(32)

In [80]:
encoder_data = torch.Tensor(encoder_data)
discriminator_data = torch.Tensor(discriminator_data)
yhat = torch.Tensor(yhat)

In [48]:
encoder_data.shape

torch.Size([32, 49, 54, 1])

In [34]:
discriminator_data.shape

torch.Size([32, 75, 54, 1])

In [108]:
reload(Encoder)
reload(Decoder)
reload(ConvSeq2Seq)
reload(Discriminator)

<module 'VAE.Discriminator' from '/home/wichen/repos/CS7643_DL/Project/VAE/Discriminator.py'>

In [54]:
st_encoder = Encoder.Encoder(3,enc_shape=[None, 20, 54, 1], enc_dim=512)

In [59]:
lt_encoder = Encoder.Encoder(3, enc_dim=527)

In [60]:
decoder = Decoder.Decoder(st_encoder)

In [65]:
d_encoder = Encoder.Encoder(32, enc_dim=512, enc_shape=[None, 75,54,1])

In [103]:
generator = ConvSeq2Seq.ConvSeq2Seq(lt_encoder, decoder, window_length=20)

In [62]:
test = generator.forward(encoder_data, discriminator_data)

encoder_output:  torch.Size([32, 527])


In [109]:
discriminator = Discriminator.Discriminator(32, d_encoder)

In [110]:
test = discriminator.forward(discriminator_data,yhat)

In [115]:
G_criterion = nn.MSELoss()
D_criterion = nn.BCELoss()

In [104]:
predicted_seq, predicted_action, generated_sample = generator.forward(encoder_data, discriminator_data)

In [89]:
expected_seq = discriminator_data[:, 50:, :, :]

In [88]:
predicted_seq.shape

torch.Size([32, 25, 54, 1])

In [90]:
expected_seq.shape

torch.Size([32, 25, 54, 1])

In [101]:
generated_sample.shape

torch.Size([32, 75, 54, 1])

In [93]:
ReconstructError = G_criterion(predicted_seq, expected_seq)

In [118]:
d_logits_real = discriminator.forward(discriminator_data, yhat)
d_logits_fake = discriminator.forward(generated_sample, yhat)

In [119]:
d_loss_real = torch.mean(D_criterion(d_logits_real, torch.ones_like(d_logits_real)))
d_loss_fake = torch.mean(D_criterion(d_logits_fake, torch.zeros_like(d_logits_fake)))
g_loss = torch.mean(D_criterion(d_logits_fake, torch.ones_like(d_logits_fake)))

In [120]:
loss_d = d_loss_real + d_loss_fake

In [121]:
loss_d

tensor(1.3958, grad_fn=<AddBackward0>)

In [122]:
g_loss

tensor(0.7648, grad_fn=<MeanBackward0>)

In [ ]:
for i in range(10):
    
    

In [ ]:
# Initialize BCELoss function
criterion = nn.BCELoss()

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [3]:
dloader.train_set[(1,'walking', 1, 'even')].shape

(1738, 54)

In [12]:
a = VAE.EncoderBase(1,2,3,4)

In [21]:
import numpy as np

In [26]:
np.random.randint( 16, 50)

23

In [22]:
np.random.choice(5,3)

array([0, 0, 4])

In [1]:
import torch

In [3]:
torch.float32

torch.float32

In [11]:
for x in range(0, 10, 2):
    print(x)

0
2
4
6
8


In [12]:
import data_utils

In [18]:
filename = '{0}/S{1}/{2}_{3}.txt'.format('./data/h3.6m/dataset', 1, 'walking', 1)
action_sequence = data_utils.readCSVasFloat(filename)

In [19]:
action_sequence.shape

(3476, 99)